Consolidating all metadata about the seams. There has been metadata in the repository, including the filepaths, and then an excel file DocOrgnizaiton in the pdf folder without filepaths but more papers. 

I believe DocOrgnizaiton was extended from the original file I created manually recording all titles from the table of contents, which seems to have papers not in the seams collection. It appears all these missing papers were removed in the metadata used in the repo. 

I have now combined the two datasets. Titles that don't have an associated pdf can just be removed by dropping missing values from that column

This notebook runs some tests to compare that the metadata is intact


Update 04-28-2021 I went through this notebook again before edx upload. Cleaned up the structure a bit. 

In [1]:
import sys

sys.path.append(os.path.join('..', 'nlp_utils'))

from nlp_utils.ocr import tesseract_ocr
import pandas as pd
import sqlite3
import os
from shutil import copyfile
import numpy as np

import git
repopath = git.Repo('.', search_parent_directories=True).working_tree_dir
data_folder = os.path.join(repopath, 'data')

con = sqlite3.connect(os.path.join(data_folder, 'seamsnlp.db'))
cursor = con.cursor()

df = pd.read_sql_query("SELECT * FROM metadata", con, index_col='ID')
df['Title'] = df['Title'].apply(str.strip)

old_pdf_folder = r'C:\Users\aspit\National Energy Technology Laboratory\MHD Lab - Documents\Publications\SEAMs Archive\Updated SEAMs'
new_pdf_folder = r'C:\Users\aspit\National Energy Technology Laboratory\MHD Lab - Documents\Publications\SEAMs'

df_meta_new  = pd.read_csv(os.path.join(new_pdf_folder, 'SEAMs_metadata.csv'), index_col=0)
df_meta_old  = pd.read_csv(os.path.join(old_pdf_folder, 'SEAMs_metadata.csv'), index_col=0)


In [2]:
#Check files in new metadata that have no filepath but do not say 'missing' 

df_meta_new_missing_filepaths = df_meta_new.where(df_meta_new['Filepath'] != df_meta_new['Filepath']).dropna(subset=['SEAM'])
df_meta_new_missing_filepaths.where(df_meta_new_missing_filepaths['File Info'] != 'Missing').dropna(subset=['SEAM'])

,SEAM,Session Name,Title,Author(s),Filepath,File Info,Year,First Author
ID,,,,,,,,


check for missing for missing files in old metadata. I removed some double spaces from these filenames/folders so now they show up missing here, but not in the new colleciton

In [3]:
for rel_fp in df_meta_old['Filepath'].dropna():
    fp = os.path.join(old_pdf_folder, rel_fp)

    fp = "\\\\?\\" + fp.replace('/', '\\')
    if not os.path.exists(fp):
        print(rel_fp)


new metadaa

In [4]:

# df_meta_new = df_meta_new.dropna(subset=['Filepath'])
# df_meta_new['Title'] = df_meta_new['Title'].apply(str.strip)

# df_meta_new['Title']

# csv_titles = df_meta_new['Title']
# csv_titles.index = np.arange(1,len(csv_titles)+1, 1)
# csv_titles

Check that databaes have consistent titles. Looks likethere are a few that are slightly different someehow, but both databases seem to have them. 

In [5]:
#Titles in new metadata but not in old metadata

non_missing_new = df_meta_new.where(df_meta_new['File Info'] != 'Missing').dropna()

for title in non_missing_new['Title']:

    num_titles = len(df_meta_old['Title'].where(df_meta_old['Title'] == title).dropna())
    if num_titles == 0:
        print(title)


In [6]:
#Titles in old metadata but not new metadata



for title in df_meta_old['Title']:

    if title in non_missing_new.index:
        num_titles = len(non_missing_new.where(non_missing_new == title).dropna())
        if num_titles == 0:
            print(title)



Check that all filepaths point to an existing file

In [7]:

filepaths = df_meta_new.dropna(subset=['Filepath'])['Filepath']
# filepaths = filepaths.apply(str.strip)

missing_files = []

for rel_fp in filepaths:
    fp = os.path.join(new_pdf_folder, rel_fp)

    fp = "\\\\?\\" + fp.replace('/', '\\')
    if not os.path.exists(fp):
        missing_files.append(rel_fp)

missing_files

[]

explicitly checking that the filepaths match. The inconsistent filepaths match with the removal of whitespace as mentioned above 

In [37]:
#I believe this was from before I changed the relative filepath strucutre...


# df_docsv_u = df_meta_old.drop_duplicates('Title').dropna(subset=['Filepath'])
# df_u = df_meta_new.drop_duplicates('Title')

# df_u['Title'] = df_u['Title'].apply(str.strip)

# for i in df_docsv_u.index[0:10]:
#     title = df_docsv_u.loc[i]['Title'].strip()
#     filepath = df_docsv_u.loc[i]['Filepath']

#     df_sel = df_u.where(df_u['Title'] == title).dropna(subset=['SEAM'])

#     if len(df_sel) == 1:
#         # print(df_sel)
#         filepath_df = df_sel['Filepath'].item()
#         if filepath_df != filepath:
#             print('----------')
#             print("Filepaths do not match")
#             print('New')
#             print(title)
#             print(filepath)
#             print('old')
#             print(filepath_df)
#             print('----------')
#     elif len(df_sel) == 0:
#         print('-----------')
#         print('Missing Title')
#         print(title)
#         print('-----------')
#     elif len(df_sel) > 1:
#         print('-----------')
#         print('multiple titles')
#         print(title)
#         print('-----------')
